In [1]:
from tkinter import filedialog
import tkinter as tk
from PIL import ImageTk, Image
import numpy as np
import matplotlib.pyplot as plt
import glob
from scipy import ndimage

class Node:
    def __init__(self, item=None):
        self.item = item
        self.link = None
        
class Circle_linked_list:
    def __init__(self):
        self.root = Node()
        self.tail = self.root
        self.current = self.root
        
    def append(self, item):
        curNode = self.root
        newNode = Node(item)
        if curNode.item == None:
            self.root = newNode
            self.tail = newNode
        else:
            self.tail.link = newNode
            newNode.link = self.root
            self.tail = newNode
    
    def setCurrent(self, item):
        curNode = self.root
        while curNode.item != item:
            curNode = curNode.link
        self.current = curNode
    
    def moveNext(self):
        self.current = self.current.link
        
    def insert(self, item):
        newNode = Node(item)
        if self.current == self.tail:
            self.current.link = newNode
            newNode.link = self.root
            self.tail = newNode
        else:
            newNode = Node(item)
            newNode.link = self.current.link
            self.current.link = newNode
            
    def delete(self, item):
        curNode = self.root
        preNode = self.root
        if self.root.item == item:
            self.root = self.root.link
            self.tail.link = self.root
        else:
            curNode = curNode.link
            while curNode.item != item:
                curNode = curNode.link
                preNode = preNode.link
            if curNode == self.tail:
                preNode.link = self.root
                self.tail = preNode
            else:
                preNode.link = curNode.link
    
    def print(self):
        curNode = self.root
        print(curNode.item)
        if curNode.link != None:
            curNode = curNode.link
        while curNode != self.root:
            print(curNode.item)
            curNode = curNode.link

class Dequeue:
    def __init__(self):
        self.dq = []
        
    def insertFirst(self, item):
        self.dq.insert(0, item)
        
    def insertLast(self, item):
        self.dq.append(item)
        
    def isEmpty(self):
        if len(self.dq) > 0:
            return False
        else:
            return True
        
    def popFirst(self):
        return self.dq.pop(0)
    
    def popLast(self):
        return self.dq.pop(-1)
    
    def peekFirst(self):
        return self.dq[0]
    
    def peekLast(self):
        return self.dq[-1]    
    
    def print(self):
        print(self.dq)
        
    def sum(self):
        return sum(self.dq)

class Window:
    def __init__(self):
        self.window=tk.Tk()
        self.window.title("Image Viewer")
        self.window.geometry("640x480")
        self.window.resizable(False, False)

        # 메뉴바를 윈도우에 추가한다.
        menubar = tk.Menu(self.window)
        menu_1 = tk.Menu(menubar, tearoff=0)
        menu_1.bind('<<MenuSelect>>')

        menu_1.add_command(label="Open", command=self.fileSelect)
        menubar.add_cascade(label="File", menu=menu_1)

        self.window.config(menu=menubar)

        # 이미지 라벨을 추가한다.
        self.imgLabel = tk.Label(self.window, width=400, height=400, relief='solid')
        self.imgLabel.pack()

        #버튼을 추가한다
        self.button = tk.Button(self.window, overrelief="solid", width=15, command=self.blur, repeatdelay=1000, repeatinterval=100, text = "blur")
        self.button.pack()
        
        #버튼을 추가한다
        self.button = tk.Button(self.window, overrelief="solid", width=15, command=self.countUP, repeatdelay=1000, repeatinterval=100, text = "next")
        self.button.pack()
        
        self.window.mainloop()
        
    def close(self):
        self.window.quit()
        self.window.destroy()
        
    def blur(self):
        image = np.array(self.image)
        
        filtered = np.zeros(image.shape)
        windowSize = 3
        gap = windowSize // 2 # 제외되는 픽셀수
        for s in range(3):
            for i in range(gap, image.shape[0] - gap):
                # 좌에서 우로 이동하고, 끝이 되면 한칸 내려서 큐를 리셋하고 진행한다.
                dq = Dequeue()
                # 초기 큐를 생성(추가순서는 빼기 쉽게 세로우선으로 추가한다.)
                for c in range(-gap, gap + 1):
                    for r in range(-gap, gap + 1): 
                        dq.insertLast(image[i+r,1+c,s])
                filtered[i,1,s] = np.median(dq.dq)

                # 생성된 큐에서 윈도우 개수 만큼 빼고. 다시 윈도우 개수 만큼 추가한다.
                for j in range(2,image.shape[1]-1):  
                    for k in range(gap,-gap-1,-1):
                        dq.popFirst() # 하나빼고 하나 넣고 ...
                        dq.insertLast(image[i+k,j+gap,s])
                    filtered[i,j,s] = np.median(dq.dq)

        self.image = Image.fromarray(filtered.astype('uint8'), 'RGB') 

        self.image_print = ImageTk.PhotoImage(self.image)
        self.imgLabel.config(image = self.image_print)
        
    def countUP(self):
        #current를 다음으로 이동시킴
        self.Link.moveNext()
        #current의 아이템 (이미지의 경로)를 a에 저장
        a = self.Link.current.item
        self.image = Image.open(a)
        print(self.Link.current.item)
        
        if self.image.size[1] > self.image.size[0]:
            hSize = int((400 * self.image.size[0] / self.image.size[1]))
            vSize = 400
        else:
            hSize = 400
            vSize = int((400 * self.image.size[1] / self.image.size[0]))

        self.image = self.image.resize((hSize,vSize), Image.ANTIALIAS)
        self.img = ImageTk.PhotoImage(self.image)
        self.imgLabel.config(image = self.img)
        
    # 파일을 선택한다.
    def fileSelect(self):
        #새로운 순환리스트를 만든다
        self.Link = Circle_linked_list()
        # 폴더를 선택하고 폴더의 경로를 selDir에 저장한다
        selDir = filedialog.askdirectory(initialdir="/", title="Select file")
        #폴더 내의 .jpg 파일을 모두 images에 배열로 저장한다
        images = glob.glob(selDir+'/*.jpg')
        #저장된 파일을 순환리스트의 아이템으로 넣는다
        for i in images:
            self.Link.append(i)
            print(i)
        self.Link.setCurrent(images[0])
        print(self.Link.current.item)
        a = self.Link.current.item

        self.image = Image.open(a)
        print(self.image.size[0], self.image.size[1])
        # 해당 이미지의 크기를 400, 400으로 resize 한다.
        if self.image.size[1] > self.image.size[0]:
            hSize = int((400 * self.image.size[0] / self.image.size[1]))
            vSize = 400
        else:
            hSize = 400
            vSize = int((400 * self.image.size[1] / self.image.size[0]))

        self.image = self.image.resize((hSize,vSize), Image.ANTIALIAS)
        self.img = ImageTk.PhotoImage(self.image)
        self.imgLabel.config(image = self.img)

a = Window()

C:/Users/김범수/Pictures/Saved Pictures\001.jpg
C:/Users/김범수/Pictures/Saved Pictures\002.jpg
C:/Users/김범수/Pictures/Saved Pictures\003.jpg
C:/Users/김범수/Pictures/Saved Pictures\004.jpg
C:/Users/김범수/Pictures/Saved Pictures\005.jpg
C:/Users/김범수/Pictures/Saved Pictures\006.jpg
C:/Users/김범수/Pictures/Saved Pictures\007.jpg
C:/Users/김범수/Pictures/Saved Pictures\008.jpg
C:/Users/김범수/Pictures/Saved Pictures\001.jpg
300 694
